In [4]:
# career_advice_bot.py

from unsloth import FastLanguageModel
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import torch

def load_model():
    model_id = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
    # meta-llama/Meta-Llama-3-8B-Instruct
    # deepseek-ai/deepseek-llm-7b-chat
    max_seq_length = 2048

    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_id,
        max_seq_length=max_seq_length,
        dtype=torch.float16,
        load_in_4bit=True,
    )

    model.eval()
    tokenizer.padding_side = "right"

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    return HuggingFacePipeline(pipeline=pipe)


def build_prompt():
    template = """
You are a professional career advice AI chatbot, specializing in helping undergraduate and postgraduate students with their career questions.  
Provide accurate, encouraging, and practical advice based on the student’s question.

Here are some examples of how you should respond:

Student: I’m about to graduate and don’t know how to write a resume. Can you help?  
Chatbot: Of course! A good resume highlights your education, internships, and skills clearly. Keep it concise—usually one page is best. I can suggest some resume templates or connect you with a career advisor for more personalized help.

Student: I’m not sure what career suits me. Any advice?  
Chatbot: That’s a common concern. You might want to try some career interest assessments to learn more about your strengths and preferences. Attending career talks and workshops can also help. If you want, I can help you book a session with a career counselor for deeper guidance.

Student: Where can I find internship opportunities?  
Chatbot: Great question! Popular internship websites include LinkedIn, Indeed, and your university’s career portal. Also, consider attending job fairs and networking with alumni. I can help you create an internship application plan if you'd like.

Student: I feel anxious about my future and don’t know what to do.  
Chatbot: It’s completely normal to feel this way. Try some relaxation techniques and talk to friends or mentors about your concerns. If you need more support, I recommend reaching out to your university’s counseling or career services.

Student: Can you tell me about the hiring process at a specific company?  
Chatbot: Detailed info about specific company hiring processes can be complex. I suggest contacting your university’s career center or your tutor for personalized advice. Meanwhile, I can help find publicly available info if you want.

Now, please answer the following question accordingly:

Student: {question}
Chatbot:"""
    return PromptTemplate.from_template(template)


def create_chain(llm, prompt_template):
    return LLMChain(llm=llm, prompt=prompt_template)


def main():
    print("🎯 Loading model, please wait...")
    llm = load_model()

    print("✅ Model loaded. Building chat chain...")
    prompt_template = build_prompt()
    qa_chain = create_chain(llm, prompt_template)

    print("🤖 Career Advice Chatbot is ready! Enter your questions (type 'exit' to quit).")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye! Wish you success in your career!")
            break

        # Use invoke() to get output only, avoid printing the whole prompt
        raw = qa_chain.invoke({"question": user_input})["text"]
        answer = raw.split("Chatbot:")[-1].strip()
        print(f"\nAI: {answer}")



if __name__ == "__main__":
    main()


🎯 Loading model, please wait...
==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 4070 Laptop GPU. Num GPUs = 1. Max memory: 7.996 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device set to use cuda:0


✅ Model loaded. Building chat chain...
🤖 Career Advice Chatbot is ready! Enter your questions (type 'exit' to quit).



You:  Is making more money the primary reason for job hopping as a software engineer?



AI: While salary is an essential factor, it's often not the only motivation for software engineers to switch jobs. Many factors come into play, including personal fulfillment, growth opportunities, company culture, and work-life balance. In fact, research suggests that employees who stay longer tend to experience higher job satisfaction. As a software engineer, focus on building meaningful relationships, developing new skills, and finding a workplace environment that aligns with your values and interests. Salary will likely follow naturally if you prioritize your long-term career goals.



You:  exit


👋 Goodbye! Wish you success in your career!
